In [19]:
import sys
sys.path.append('../')

from utils.GetData import Data
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.feature_selection import mutual_info_regression
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [20]:
PATH = '../in/train.csv'
df = pd.read_csv(PATH, index_col='Id')

In [21]:
data = Data()

In [22]:

LotShape = ['IR3','IR2','IR1','Reg']
Utilities = ['ELO','NoSeWa','NoSewr','AllPub']
LandSlope = ['Sev','Mod','Gtl']
HouseStyle = ['1Story','1.5Fin','1.5Unf','2Story','2.5Fin','2.5Unf','SFoyer','SLvl'] # posible oh
ExterQual = ['Po','Fa','TA','Gd','Ex']
ExterCond = ['Po','Fa','TA','Gd','Ex']
BsmtQual = ['NA','Po','Fa','TA','Gd','Ex'] 
BsmtCond = ['NA','Po','Fa','TA','Gd','Ex'] 
BsmtExposure = ['NA','No','Mn','Av','Gd']
BsmtFinType1 = ['NA','Unf','LwQ','Rec','BLQ','ALQ','GLQ'] 
BsmtFinType2 = ['NA','Unf','LwQ','Rec','BLQ','ALQ','GLQ'] 
HeatingQC = ['Po','Fa','TA','Gd','Ex']
CentralAir = ['N','Y']
Electrical = ['Mix','FuseP','FuseF','FuseA','SBrkr']  
KitchenQual = ['Po','Fa','TA','Gd','Ex']
Functional = ['Sal','Sev','Maj2','Maj1','Mod','Min2','Min1','Typ']
FireplaceQu = ['NA','Po','Fa','TA','Gd','Ex'] 
GarageType = ['NA','Detchd','CarPort','BuiltIn','Basment','Attchd','2Types'] 
GarageFinish = ['NA','Unf','RFn','Fin'] 
GarageQual = ['NA','Po','Fa','TA','Gd','Ex'] 
GarageCond = ['NA','Po','Fa','TA','Gd','Ex']
PavedDrive = ['N','P','Y']
PoolQC = ['NA','Fa','TA','Gd','Ex'] 
Fence = ['NA','MnWw','GdWo','MnPrv','GdPrv']

# fill Electrical na values with 'most frecuente'
# fil GarageYrBlt na values with most frecuente
# change value None of MasVnrType to NA

col_fill_NA = ['Alley','MasVnrType','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','PoolQC','Fence','MiscFeature']
col_fill_0 = ['LotFrontage','MasVnrArea','BsmtFullBath','BsmtHalfBath','BsmtFinSF2','GarageArea','BsmtFinSF1','GarageCars','TotalBsmtSF','BsmtUnfSF']
col_fill_mf = ['MSZoning','Electrical','GarageYrBlt','Functional','Utilities','Exterior2nd','Exterior1st','SaleType','KitchenQual']

ordinal_cols_categories = [LotShape,Utilities,LandSlope, HouseStyle,ExterQual, ExterCond, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, HeatingQC, CentralAir, Electrical, KitchenQual, Functional, FireplaceQu, GarageType, GarageFinish, GarageQual, GarageCond, PavedDrive, PoolQC, Fence]
ordinal_cols = ['LotShape','Utilities','LandSlope', 'HouseStyle','ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence']

# antes de ejecutar oh, rellenar los na de MiscFeature y Alley con 'NA'
oh_cols = ['MSZoning','Street','Alley','LandContour','LotConfig','Neighborhood', 'Condition1','Condition2','BldgType','RoofStyle','RoofMatl','Exterior1st','Exterior2nd', 'MasVnrType','Foundation','Heating','MiscFeature','SaleType','SaleCondition']

In [23]:
df.isna().sum().sort_values(ascending=False)[:20]

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
FireplaceQu      690
LotFrontage      259
GarageYrBlt       81
GarageCond        81
GarageType        81
GarageFinish      81
GarageQual        81
BsmtExposure      38
BsmtFinType2      38
BsmtCond          37
BsmtQual          37
BsmtFinType1      37
MasVnrArea         8
MasVnrType         8
Electrical         1
MSSubClass         0
dtype: int64

In [24]:
imputer_N = SimpleImputer(strategy='constant',fill_value='NA')
df[col_fill_NA] = pd.DataFrame(imputer_N.fit_transform(df[col_fill_NA]), columns=col_fill_NA)

imputer_0 = SimpleImputer(strategy='constant',fill_value=0)
df[col_fill_0] = pd.DataFrame(imputer_0.fit_transform(df[col_fill_0]), columns=col_fill_0)

imputer_mf = SimpleImputer(strategy='most_frequent')
df[col_fill_mf] = pd.DataFrame(imputer_mf.fit_transform(df[col_fill_mf]), columns=col_fill_mf)

df['MasVnrType'].replace('None', 'NA', inplace=True)



In [25]:
ordinal = OrdinalEncoder(categories= ordinal_cols_categories)
df[ordinal_cols] = pd.DataFrame(ordinal.fit_transform(df[ordinal_cols]), columns=ordinal_cols)

ValueError: Found unknown categories [nan] in column 1 during fit

In [ ]:
oh_encoder = OneHotEncoder(handle_unknown='ignore')
df_oh = pd.DataFrame(oh_encoder.fit_transform(df[oh_cols]).toarray())

In [ ]:
df.shape

(1460, 81)

In [ ]:
df.drop(oh_cols, axis=1, inplace=True)
X = pd.concat([df,df_oh], join='inner', axis=1)

In [ ]:
X.shape

(1460, 209)

In [ ]:
X.head()

,Id,MSSubClass,LotFrontage,LotArea,LotShape,Utilities,LandSlope,HouseStyle,OverallQual,OverallCond,...,137,138,139,140,141,142,143,144,145,146
0,1,60,65.0,8450,3.0,3.0,2.0,3.0,7,5,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2,20,80.0,9600,3.0,3.0,2.0,0.0,6,8,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,3,60,68.0,11250,2.0,3.0,2.0,3.0,7,5,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,4,70,60.0,9550,2.0,3.0,2.0,3.0,7,5,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,5,60,84.0,14260,2.0,3.0,2.0,3.0,8,5,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:

X = X.drop(columns=['SalePrice','Id'], axis=1)

discrete_features = X.dtypes == int
y = df['SalePrice']

In [ ]:
X.shape

(1460, 207)

In [ ]:
mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)

In [ ]:
mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
mi_scores = mi_scores.sort_values(ascending=False)

In [ ]:
import numpy as np

In [ ]:
def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks, fontsize=10)
    plt.title("Mutual Information Scores")


plt.figure(dpi=100, figsize=(8, 15))
plot_mi_scores(mi_scores)

In [ ]:
model = RandomForestRegressor()

parameters = {
    'n_estimators': [1, 30],
    'criterion': ['squared_error','absolute_error'],
    'max_depth': [1, 30]
}

rank_estimator = GridSearchCV(model, parameters, cv=5, scoring='neg_mean_absolute_error')
rank_estimator.fit(X, y)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'criterion': ['squared_error', 'absolute_error'],
                         'max_depth': [1, 30], 'n_estimators': [1, 30]},
             scoring='neg_mean_absolute_error')

In [ ]:
rank_estimator.best_params_

{'criterion': 'absolute_error', 'max_depth': 30, 'n_estimators': 30}

In [ ]:
pred_example = pd.read_csv('../in/sample_submission.csv')

In [ ]:
test = pd.read_csv('../in/test.csv')

In [ ]:
test[col_fill_NA] = pd.DataFrame(imputer_N.transform(test[col_fill_NA]), columns=col_fill_NA)

test[col_fill_0] = pd.DataFrame(imputer_0.transform(test[col_fill_0]), columns=col_fill_0)

test[col_fill_mf] = pd.DataFrame(imputer_mf.transform(test[col_fill_mf]), columns=col_fill_mf)


test['MasVnrType'].replace('None', 'NA', inplace=True)


In [ ]:
test[ordinal_cols] = pd.DataFrame(ordinal.transform(test[ordinal_cols]), columns=ordinal_cols)

In [ ]:
test_oh = pd.DataFrame(oh_encoder.transform(test[oh_cols]).toarray())

In [ ]:
test.drop(oh_cols, axis=1, inplace=True)

In [ ]:
X_test = pd.concat([test,test_oh], join='inner', axis=1)

In [ ]:
X_test.drop('Id', axis=1, inplace=True)

In [ ]:
predict = pd.DataFrame(rank_estimator.predict(X_test), columns=['SalePrice'])
predict['Id'] = index=pred_example.Id
predict = predict.set_index('Id')
predict.to_csv('../out/one-hot.csv')
predict.head()

,SalePrice
Id,
1461,128728.333333
1462,150738.333333
1463,174533.333333
1464,183219.666667
1465,194366.666667
